In [ ]:
dataset_name = "CSTR"

# Imports, load data , training functions and database creation

## Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, errno
import networkx as nx
from networkx.algorithms import bipartite
import json
import yaml
import ast
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
import pickle
import json

import tensorflow as tf

In [ ]:
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 32.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import torch_geometric
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.utils.convert import from_networkx
from torch_geometric.nn import SAGEConv, GATConv, Linear, to_hetero

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device

## load data

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [ ]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,126.txt,Rhetorical (Rhet) is a programming / knowledge...,ArtificiallIntelligence,"[(rhetorical rhet, 0.7475), (rhet programming,...","[(rhetorical rhet programming, 0.8299), (rhet ...","[(rhetorical rhet programming, 0.8299), (rheto...","[-0.022039127, 0.02312275, -0.076888025, -0.06...","[[0.0059065777, 0.040003918, 0.0011322909, -0....","[[-0.045958184, 0.03794197, -0.027850023, -0.0...","[[-0.045958184, 0.03794197, -0.027850023, -0.0...","[[0.0026859022, 0.0066884025, -0.07187084, -0...."
1,1,5.txt,Reduction is the operation of transforming a p...,ArtificiallIntelligence,"[(parsing complexity, 0.5977), (transformed gr...","[(complexity resulting grammar, 0.675), (minim...","[(complexity resulting grammar, 0.675), (minim...","[0.074621424, -0.007094671, 0.051812325, -0.03...","[[0.0426418, -0.013719074, 0.02755835, -0.0075...","[[0.07436234, -0.0029446871, 0.052716985, 0.03...","[[0.07436234, -0.0029446871, 0.052716985, 0.03...","[[0.008135828, 0.0732408, 0.030002978, -0.0219..."
2,2,48.txt,"For years, researchers have used knowledge-int...",ArtificiallIntelligence,"[(disambiguating parsing, 0.7122), (syntactic ...","[(techniques disambiguating parsing, 0.7356), ...","[(techniques disambiguating parsing, 0.7356), ...","[-0.014407578, -0.077485226, 0.083884224, 0.01...","[[0.014474175, -0.027266933, 0.07052159, -0.02...","[[0.022918042, -0.01977642, 0.047498055, -0.01...","[[0.022918042, -0.01977642, 0.047498055, -0.01...","[[0.013398974, -0.025847042, 0.0290076, 0.0357..."
3,3,81.txt,Proceedings of a workshop held in conjunction ...,ArtificiallIntelligence,"[(machine learning, 0.6059), (tahoe city, 0.39...","[(machine learning july, 0.663), (conference m...","[(machine learning july, 0.663), (machine lear...","[-0.036386, -0.08516384, 0.06518213, 0.0014036...","[[-0.024391863, 0.0032444268, 0.054267664, -0....","[[-0.075079046, -0.10737129, 0.06574192, 0.002...","[[-0.075079046, -0.10737129, 0.06574192, 0.002...","[[-0.036386, -0.08516384, 0.06518213, 0.001403..."
4,4,25.txt,The Medication Advisor is the latest project o...,ArtificiallIntelligence,"[(medication advisor, 0.5975), (conversational...","[(medications medication advisor, 0.62), (cons...","[(medications medication advisor, 0.62), (cons...","[-0.050106738, -0.013832041, 0.03398316, -0.02...","[[-0.030672388, 0.007967623, -0.023997275, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0...","[[-0.020701267, 0.003092026, -0.024871623, 0.0...","[[-0.05715183, -0.03530269, 0.015551494, -0.00..."


In [ ]:
classes = df["class"].unique()
print(classes)
class_number = len(classes)
print(class_number)

['ArtificiallIntelligence' 'Robotics' 'Systems' 'Theory']
4


# Functions


In [ ]:
def define_graph(model_name,rotulated,iteration):
    if (rotulated != 80):
        with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'_'+str(iteration)+'.pkl', 'rb') as f:
            aux = pickle.load(f)
            train_mask, val_mask, test_mask = aux[0],aux[1],aux[2]
    else:
        with open('/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/'+dataset_name+'/masks/mask_rot'+str(rotulated)+'%_'+str(iteration)+'.pkl', 'rb') as f:
            aux = pickle.load(f)
            train_mask, val_mask, test_mask = aux[0],aux[1],aux[2]

    # Novo objeto para representar edges
    new_edges = np.copy(edges)

    # A conversion on vertices is needed:
    # we must add documents and contexts nodes in the same integer sequence
    # since the coarcening algorithm does not diferentiate nodes by their bipartide placement
    number_doc_nodes = len(document_nodes)
    new_edges[1] = new_edges[1] + number_doc_nodes

    # Selecting training instances to be excluded from coarsening
    mask_excluding_array = [i for i,x in enumerate(train_mask) if x==True]

    # adjustion new_egdges representation
    new_edges = new_edges.T

    # excluindo as mascaras
    exc_edges = np.array([[0,0]])
    for exc in mask_excluding_array:
        # Exclui a linha que possui um valor expecifico em sua coluna
        # col -> coluna para verificar o valor que queremos exluir
        # val -> valor que desejamos excluir
        # out = arr[~(arr[:,col] == val),:]
        exc_edges = np.concatenate((exc_edges, np.array(new_edges[(new_edges[:,0] == exc),:])), axis=0)
        new_edges = new_edges[~(new_edges[:,0] == exc),:]
    exc_edges = np.delete(exc_edges, 0, 0)

    # saving first file
    json_dict = {"edges":len(new_edges),"vertices":[len(document_nodes),len(context_nodes)]}
    json_object = json.dumps(json_dict, indent=4)


    # saving second file the .ncol
    lines = [str(tupla[0])+' '+str(tupla[1])+' '+'1.0\n' for tupla in new_edges]


    # Saving separetly for easy downloads
    with open("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/easy_download/"+model_name+"_"+str(rotulated)+'_'+str(iteration)+".json", 'w') as f:
        f.write(json_object)
        f.close()

    with open("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/easy_download/"+model_name+"_"+str(rotulated)+'_'+str(iteration)+".ncol", 'w') as f:
        f.writelines(lines)
        f.close()

    # Saving excluded edges for later reading them to the graph
    with open("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated)+"/exc_edges_"+model_name+"_"+str(rotulated)+'_'+str(iteration), 'wb') as handle:
        pickle.dump(exc_edges, handle)

In [ ]:
#Without easy downloads
# with open("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated)+"/"+model_name+"_"+str(rotulated)+'_'+str(iteration)+".json", 'w') as f:
#     f.write(json_object)
#     f.close()

# with open("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated)+"/"+model_name+"_"+str(rotulated)+'_'+str(iteration)+".ncol", 'w') as f:
#     f.writelines(lines)
#     f.close()

--------------------------------------
# Hetrogenous Graph for Keyphrase = 2
--------------------------------------

In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase2"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 299
number of context nodes: 1314
number of shared contexts: 181
number of direct edges (first dimension): 1495
number of direct edges (second dimension): 1495


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [ ]:
# Show documents nodes
document_nodes

tensor([[-0.0220,  0.0231, -0.0769,  ...,  0.1419,  0.1079, -0.0344],
        [ 0.0746, -0.0071,  0.0518,  ...,  0.0495,  0.0877,  0.0238],
        [-0.0144, -0.0775,  0.0839,  ...,  0.0047,  0.0209,  0.0116],
        ...,
        [ 0.0063, -0.0461,  0.0246,  ...,  0.0435, -0.0737, -0.0371],
        [-0.0916,  0.0071, -0.0406,  ...,  0.0551,  0.0544, -0.0292],
        [-0.0790,  0.0077, -0.0081,  ...,  0.0168,  0.0037,  0.0190]])

In [ ]:
# Quantity of document nodes
len(document_nodes)

299

In [ ]:
# Show context nodes
context_nodes

tensor([[ 0.0059,  0.0400,  0.0011,  ...,  0.0658,  0.0987, -0.1089],
        [-0.0788, -0.0112, -0.0691,  ...,  0.0927,  0.0427, -0.0612],
        [ 0.0232,  0.0165, -0.0164,  ...,  0.1524,  0.0515, -0.0367],
        ...,
        [ 0.0433, -0.0433, -0.0125,  ...,  0.0168,  0.0225, -0.0429],
        [-0.0242,  0.0415,  0.0377,  ...,  0.0636,  0.0278, -0.0133],
        [-0.0937,  0.0558,  0.0037,  ...,  0.0429,  0.0639,  0.0129]])

In [ ]:
# Quantity of context nodes
len(context_nodes)

1314

In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

34630
34630


In [ ]:
# showing edges
print(edges)

tensor([[   0,    0,    0,  ...,  298,  298,  298],
        [   0,    1,    2,  ..., 1148, 1251, 1232]])


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['ArtificiallIntelligence' 'Robotics' 'Systems' 'Theory']


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'ArtificiallIntelligence': 0, 'Robotics': 1, 'Systems': 2, 'Theory': 3} 



<ipython-input-26-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
len(y)

299

## Saving graph

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
with open('/content/drive/MyDrive/mestrado_v2/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes_text,context_nodes_text,y,class_number,classes]
    pickle.dump(aux, f)

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/easy_download")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/drop_and_move")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

## Removing Masks

### rotulated base = 1 ❎

In [ ]:
rotulated = 1
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


### rotulated base = 5 ❎

In [ ]:
rotulated = 5
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 10 ❎

In [ ]:
rotulated = 10
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 20 ❎

In [ ]:
rotulated = 20
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 30 ❎

In [ ]:
rotulated = 30
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

--------------------------------------
# Hetrogenous Graph for Keyphrase = 3
--------------------------------------

In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase3"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 299
number of context nodes: 1422
number of shared contexts: 73
number of direct edges (first dimension): 1495
number of direct edges (second dimension): 1495


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [ ]:
# Show documents nodes
document_nodes

tensor([[-0.0220,  0.0231, -0.0769,  ...,  0.1419,  0.1079, -0.0344],
        [ 0.0746, -0.0071,  0.0518,  ...,  0.0495,  0.0877,  0.0238],
        [-0.0144, -0.0775,  0.0839,  ...,  0.0047,  0.0209,  0.0116],
        ...,
        [ 0.0063, -0.0461,  0.0246,  ...,  0.0435, -0.0737, -0.0371],
        [-0.0916,  0.0071, -0.0406,  ...,  0.0551,  0.0544, -0.0292],
        [-0.0790,  0.0077, -0.0081,  ...,  0.0168,  0.0037,  0.0190]])

In [ ]:
# Quantity of document nodes
len(document_nodes)

299

In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0460,  0.0379, -0.0279,  ...,  0.1318,  0.0712, -0.0755],
        [-0.0522, -0.0016, -0.0994,  ...,  0.0759,  0.0378, -0.0089],
        [ 0.0388, -0.0221,  0.0033,  ...,  0.1689,  0.0871, -0.0248],
        ...,
        [-0.0145,  0.0006,  0.0273,  ...,  0.0439,  0.0060, -0.0276],
        [ 0.0247, -0.0422, -0.0180,  ...,  0.0323,  0.0403, -0.0162],
        [-0.0344,  0.0151, -0.0252,  ..., -0.0531,  0.0236,  0.0178]])

In [ ]:
# Quantity of context nodes
len(context_nodes)

1422

In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

34630
34630


In [ ]:
# showing edges
print(edges)

tensor([[   0,    0,    0,  ...,  298,  298,  298],
        [   0,    1,    2,  ..., 1298, 1244, 1373]])


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['ArtificiallIntelligence' 'Robotics' 'Systems' 'Theory']


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'ArtificiallIntelligence': 0, 'Robotics': 1, 'Systems': 2, 'Theory': 3} 



<ipython-input-54-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
len(y)

299

## Saving graph

In [ ]:
 # cria dirrotio para keyphrase
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
with open('/content/drive/MyDrive/mestrado_v2/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes_text,context_nodes_text,y,class_number,classes]
    pickle.dump(aux, f)

In [ ]:
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/easy_download")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/drop_and_move")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

## Removing Masks

### rotulated base = 1 ❎

In [ ]:
rotulated = 1
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


### rotulated base = 5 ❎

In [ ]:
rotulated = 5
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 10 ❎

In [ ]:
rotulated = 10
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 20 ❎

In [ ]:
rotulated = 20
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 30 ❎

In [ ]:
rotulated = 30
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

--------------------------------------
# Hetrogenous Graph for Keyphrase = 23
--------------------------------------

In [ ]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase23"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [ ]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [ ]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 299
number of context nodes: 1392
number of shared contexts: 103
number of direct edges (first dimension): 1495
number of direct edges (second dimension): 1495


In [ ]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [ ]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [ ]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [ ]:
# Show documents nodes
document_nodes

tensor([[-0.0220,  0.0231, -0.0769,  ...,  0.1419,  0.1079, -0.0344],
        [ 0.0746, -0.0071,  0.0518,  ...,  0.0495,  0.0877,  0.0238],
        [-0.0144, -0.0775,  0.0839,  ...,  0.0047,  0.0209,  0.0116],
        ...,
        [ 0.0063, -0.0461,  0.0246,  ...,  0.0435, -0.0737, -0.0371],
        [-0.0916,  0.0071, -0.0406,  ...,  0.0551,  0.0544, -0.0292],
        [-0.0790,  0.0077, -0.0081,  ...,  0.0168,  0.0037,  0.0190]])

In [ ]:
# Quantity of document nodes
len(document_nodes)

299

In [ ]:
# Show context nodes
context_nodes

tensor([[-0.0460,  0.0379, -0.0279,  ...,  0.1318,  0.0712, -0.0755],
        [ 0.0059,  0.0400,  0.0011,  ...,  0.0658,  0.0987, -0.1089],
        [-0.0788, -0.0112, -0.0691,  ...,  0.0927,  0.0427, -0.0612],
        ...,
        [-0.0145,  0.0006,  0.0273,  ...,  0.0439,  0.0060, -0.0276],
        [-0.0154,  0.0035,  0.0364,  ...,  0.0337,  0.0217, -0.0218],
        [ 0.0247, -0.0422, -0.0180,  ...,  0.0323,  0.0403, -0.0162]])

In [ ]:
# Quantity of context nodes
len(context_nodes)

1392

In [ ]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

34630
34630


In [ ]:
# showing edges
print(edges)

tensor([[   0,    0,    0,  ...,  298,  298,  298],
        [   0,    1,    2,  ..., 1207, 1270, 1219]])


#### Class Labels

In [ ]:
# All different classes
print(df["class"].unique())

['ArtificiallIntelligence' 'Robotics' 'Systems' 'Theory']


In [ ]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'ArtificiallIntelligence': 0, 'Robotics': 1, 'Systems': 2, 'Theory': 3} 



<ipython-input-82-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [ ]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [ ]:
len(y)

299

## Saving graph

In [ ]:
 # cria diretorio para keyphrase
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
with open('/content/drive/MyDrive/mestrado_v2/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes_text,context_nodes_text,y,class_number,classes]
    pickle.dump(aux, f)

In [ ]:
 # cria diretorio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/easy_download")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+"/drop_and_move")
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

## Removing Masks

### rotulated base = 1 ❎

In [ ]:
rotulated = 1
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


### rotulated base = 5 ❎

In [ ]:
rotulated = 5
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 10 ❎

In [ ]:
rotulated = 10
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 20 ❎

In [ ]:
rotulated = 20
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)

### rotulated base = 30 ❎

In [ ]:
rotulated = 30
 # cria dirrotio para o grafo especifico
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name+'/'+keyphrase+"/graphs_rotulated_"+str(rotulated))
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [ ]:
for iteration in range(10):
    define_graph(model_name,rotulated,iteration)